In [1]:
#imports

import numpy as np
import matplotlib.pyplot as plt
import glob
from os import path

In [ ]:
#data
#placeholder
data = [0.6, 0.5, 0.7]

In [ ]:
plt.plot(data)
plt.show()

In [13]:
def metis_partition_file_converter(inp, out):
    import sys

    #inp = sys.argv[1]
    #out = sys.argv[2]

    partitions = {}

    def add(el, idx):
        if idx not in partitions:
            partitions[idx] = [el]
        else:
            partitions[idx].append(el)


    with open(inp, "r+") as f:
        el = 0
        for line in f:
            idx = int(line.strip())
            add(el, idx)
            el += 1

    with open(out, "w+") as f:
        idxs = list(partitions.keys())
        idxs.sort()
        assert(idxs[0] <= idxs[-1])
        for ix in idxs:
            f.write(" ".join([str(e) for e in partitions[ix]]) + "\n")

In [26]:
#!for graph in graphs/*.graph; do echo "time start"; time timeout 5m ./a.out  --G_phi=0.05 --H_phi=0.4 --vol=1 --h_ratio=0. -f $graph | tee "$graph.out" ; echo "time end"; done
#!SHELL=/bin/bash

import glob


def run_decomp(graph_files):
    for graph in graph_files:
        print(graph)

        #run decomposition on all graphs with time limit
        out_path = graph + ".out"
        !time -p timeout 15m ./a.out  --G_phi=0.025 --H_phi=0.4 --vol=1 --h_ratio=0. -f $graph > "$out_path"

        #TODO return code
        if not glob.glob(graph + "cut.txt"):
            print("decomp on", graph, "did not finish in time")
            continue

        #how many clusters, k, did we get?
        with open(out_path) as f:
            lns = f.readlines()
            cluster_line = next(l for l in lns if "n clusters" in l)
            cluster_line.strip("\n")
            n_clusters = int(cluster_line.split(";")[1])

        if n_clusters == 1:
            print("No cut found, continue")
            continue

        print("n clusters found", n_clusters)
        #run metis on k
        metis_stdio_path = graph + ".out.metis"
        !/usr/local/bin/gpmetis -ufactor=1000 $graph $n_clusters -contig >  $metis_stdio_path

        metis_file = graph + ".part." + str(n_clusters)
        decomp_file = graph + "cut.txt"
        
        metis_partition_file_converter(metis_file, metis_file)

        rw_graph      = graph + ".row_whole"
        rw_file_ours  = graph + ".rw_ours"
        rw_file_metis = graph + ".rw_metis"

        !./a.out  --G_phi=0.025 --H_phi=0.4 --vol=1 --h_ratio=0. -f "$graph" --only_test_expander  > "$rw_graph"
        !./a.out  --G_phi=0.025 --H_phi=0.4 --vol=1 --h_ratio=0. -f "$graph" --only_test_expander -p "$decomp_file" > "$rw_file_ours"
        !./a.out  --G_phi=0.025 --H_phi=0.4 --vol=1 --h_ratio=0. -f "$graph" --only_test_expander -p "$metis_file"  > "$rw_file_metis"
    
        bname = os.path.basename(graph).split(".")[0]
        !mkdir -p results/"$bname"
        for f in glob.glob("".join(graph.split(".")[:-1]) + "*"):
            if f != graph:
                !mv $f results/"$bname"/


#graph_files = glob.glob("graphs/*.graph")
#graph_files.sort()
#run_decomp(graph_files)


In [3]:
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances
import scipy.sparse as sparse


n = 3
matrix = [[1, 1, 0], [0, 1, 1], [1, 0, 1]]
matrix = np.array(matrix)

uniform = [ 1/n for _ in range(n)]
uniform = np.array(uniform)

walk    = [1] + [0 for _ in range(n - 1)]
walk    = np.array(walk)

matrix = normalize(matrix, axis=1, norm='l1')


distance = pairwise_distances((matrix @ walk).reshape(-1, 1), uniform.reshape(-1, 1))
distance = pairwise_distances((matrix @ (matrix @ (matrix @ walk))).reshape(-1, 1), uniform.reshape(-1, 1))
steps = 0
threshold = 0.01
while np.sum(pairwise_distances(walk.reshape(-1, 1), uniform.reshape(-1, 1))) > threshold:
    walk = matrix @ walk
    print(walk)
    print( np.sum(pairwise_distances(walk.reshape(-1, 1), uniform.reshape(-1, 1))))
    steps += 1

print("steps", steps)


def test_convergence(matrix, threshold):
    np.set_printoptions(threshold=np.inf)    
    n = len(matrix)
    matrix = np.array(matrix)
    uniform = np.array([1./n for _ in range(n)])
    walk = np.array([1] + [0 for _ in range(n - 1)]) #Seed?

    matrix = np.array([[e if i !=j else np.count_nonzero(subl == 1) for i, e in enumerate(subl)] for j, subl in enumerate(matrix)])
    matrix = normalize(matrix, axis=0, norm='l1')
    print(matrix)
    steps = 0
    dist = np.sum(pairwise_distances(walk.reshape(-1, 1), uniform.reshape(-1, 1))) / n
    while dist > threshold:
        walk = matrix @ walk
        steps += 1
        dist = np.sum(pairwise_distances(walk.reshape(-1, 1), uniform.reshape(-1, 1))) / n

        if steps % 100 == 0:
            print(walk)
            #print(dist)
        if steps > 500:
            break
        print(steps)
    return steps



[0.5 0.  0.5]
2.0
[0.25 0.25 0.5 ]
1.0000000000000002
[0.25  0.375 0.375]
0.4999999999999996
[0.3125 0.375  0.3125]
0.25000000000000155
[0.34375 0.34375 0.3125 ]
0.12500000000000178
[0.34375  0.328125 0.328125]
0.0624999999999969
[0.3359375 0.328125  0.3359375]
0.031250000000012434
[0.33203125 0.33203125 0.3359375 ]
0.01562500000001421
[0.33203125 0.33398438 0.33398438]
0.007812499999975132
steps 9


In [23]:
import numpy as np
import scipy.sparse as sparse

def graph_and_cut_to_numpy(gf, cf):

    graph = {}
    with open(gf, "r") as f:
        first_row = f.readline().strip("\n").split(" ")
        while [] in first_row:
            first_row.remove([])
        n, e = [int(e) for e in first_row]
        for i, row in enumerate(f.readlines()):
            graph[i] = []
            for v in row.strip("\n").split(" "):
                if v == "":
                    continue
                assert(int(v))
                graph[i].append(int(v))
                try:
                    graph[int(v)].append(i)
                except KeyError:
                    graph[int(v)] = [i]

    clusters = []
    with open(cf, "r") as f:
        for i, row in enumerate(f.readlines()):
            clusters.append([int(e) for e in row.strip("\n").split(" ") if e != ""])

    return graph, clusters

graph, cuts = graph_and_cut_to_numpy("graphs/add20.graph", "old_results/add20.graphcut.txt")

subgs = []
for cut in cuts:

    v_set = set()
    for el in cut:
        v_set.add(int(el))

    #print(v_set)
    n = len(v_set)

    subg = {k:[] for k in cut}
    for k in graph:
        for v in graph[k]:
            assert(int(v))
            if v in v_set and k in v_set:
                subg[k].append(v)
    #print 
    print(" ".join([str((e, subg[e])) for e in subg])[:10])
    #rekey
    rekey_dict = {v:i for i, v in enumerate(subg)}
    rekey_reverse= {i:v for i, v in enumerate(subg)}

    new_subg = [[0 for _ in range(n)] for _ in range(n)]
    for k in subg: 
        for v in subg[k]:
            new_subg[rekey_dict[k]][rekey_dict[v]] = 1
    
    test_convergence(new_subg, 0.01)

    




(0, [6, 14
n: 2395
e 20913
start iterate


In [ ]:
%%javascript
from IPython.display import Javascript, display
#display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))


Javascript('IPython.notebook.execute_cells_below()')
#button = w.Button(button_style='info',description="Execute `Report")
#button.on_click(run_all)
#display(button)

In [4]:

import numpy as np
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances
import scipy.sparse as sparse

"""
n = 3
matrix = [[1, 1, 0], [0, 1, 1], [1, 0, 1]]
matrix = np.array(matrix)

uniform = [ 1/n for _ in range(n)]
uniform = np.array(uniform)

walk    = [1] + [0 for _ in range(n - 1)]
walk    = np.array(walk)

matrix = normalize(matrix, axis=1, norm='l1')

distance = pairwise_distances((matrix @ walk).reshape(-1, 1), uniform.reshape(-1, 1))
distance = pairwise_distances((matrix @ (matrix @ (matrix @ walk))).reshape(-1, 1), uniform.reshape(-1, 1))
steps = 0
threshold = 0.01
while np.sum(pairwise_distances(walk.reshape(-1, 1), uniform.reshape(-1, 1))) > threshold:
    walk = matrix @ walk
    print(walk)
    print( np.sum(pairwise_distances(walk.reshape(-1, 1), uniform.reshape(-1, 1))))
    steps += 1

print("steps", steps)
"""

def test_convergence(matrix, threshold):
    #np.set_printoptions(threshold=np.inf)    
    n = len(matrix)
    assert(len(matrix[0]) == n)
    print("n:", n)
    #matrix = np.array(matrix)
    e = sum([sum(row) for row in matrix])
 
    print("e", e)

    colsums = [sum([matrix[i][j] for j in range(n)]) for i in range(n)]
    uniform = np.array([colsums[i]/e for i in range(n)], dtype=np.float64)
    #uniform = np.array([1] + [0 for _ in range(n - 1)]) #np.array([sum([e for e in matrix[i]])/e for i in range(n)])
    assert(0.99 <=sum(uniform) <= 1.01)
    walk = np.array([1.] + [0 for _ in range(n - 1)], np.float64) #Seed?
    #walk = np.array([1./n for _ in range(n)])

    sums = [sum(matrix[i]) for i in range(n)]
    for s in sums:
        pass #assert(s > 0)
    matrix_with_drift = [[e/sums[j] for i, e in enumerate(subl)] for j, subl in enumerate(matrix)]

    assert(len(matrix_with_drift) == n)
    assert(len(matrix_with_drift[0]) == n)

    for i in range(n):
        matrix_with_drift[i][i] = sum(matrix_with_drift[i])
        assert(matrix_with_drift[i][i] > 0)
        cnt = sum(matrix_with_drift[i])
        matrix_with_drift[i] = [m/cnt for m in matrix_with_drift[i]]
        assert(0.99 <= sum(matrix_with_drift[i]) <= 1.01)

    matrix = matrix_with_drift
    matrix = np.array(matrix, np.float64)

    dist = np.sum(pairwise_distances(walk.reshape(-1, 1), uniform.reshape(-1, 1))) / n
    steps = 0
    print("start iterate")

    while dist > threshold and steps < len(matrix):
        walk = np.matmul(matrix.T, walk.T)
        assert(0.99 < sum(walk) < 1.01)
        assert(0.99 < sum(uniform) < 1.01)
        steps += 1
        dist = np.linalg.norm(walk - uniform, 2)

    s_matrix = sparse.coo_matrix(matrix)
    eigenvecs, _ = sparse.linalg.eigs(s_matrix, k=2)
    print("eigenvecs sample", eigenvecs[:5])
    print("first two eigvenvals", eigenvecs[eigenvecs.argsort()[-2:][::-1]])
    #print("lambda_2", np.linalg.eig(matrix)[0])

    print("dist", dist)
    return steps



import numpy as np

def graph_and_cut_to_numpy(gf, cf):

    graph = {}
    with open(gf, "r") as f:
        first_row = f.readline().strip("\n").split(" ")
        while [] in first_row:
            first_row.remove([])
        n, e = [int(e) for e in first_row]
        for i, row in enumerate(f.readlines()):
            graph[i] = []
            for v in row.strip("\n").split(" "):
                if v == "":
                    continue
                assert(int(v))
                graph[i].append(int(v))
                try:
                    graph[v].append(int(i))
                except KeyError:
                    graph[v] = [int(i)]

    clusters = []
    with open(cf, "r") as f:
        n_counter = 0
        e_counter = 0
        for i, row in enumerate(f.readlines()):
            clusters.append([int(e) for e in row.strip("\n").split(" ") if e != ""])
            #print(clusters[-1])
            n_counter += 1
        e_counter += len(clusters[-1])


    return graph, clusters


def test():
    graph, cuts = graph_and_cut_to_numpy("graphs/add32.graph", "old_results/add32.graphcut.txt")
    #graph, cuts = graph_and_cut_to_numpy("graphs/whitaker3.graph", "old_results/whitaker3.graph.part.94.chaco")
    subgs = []

    for clidx, cut in enumerate(cuts):

        v_set = set()
        for el in cut:
            v_set.add(int(el))

        #print(v_set)
        n = len(v_set)

        subg = {k:[] for k in cut}
        #print("first element in cut", cut[0])
        for k in graph:
            for v in graph[k]:
                if v in v_set and k in v_set:
                    subg[k].append(v)

        rekey_dict    = {v:i for i, v in enumerate(sorted(list(subg.keys())))}
        rekey_reverse = {i:v for i, v in enumerate(sorted(list(subg.keys())))}

        success = True
        new_subg = [[0 for _ in range(n)] for _ in range(n)]
        for k in subg: 
            #assert(len(subg[k]) > 0)
            if len(subg[k]) == 0:
                print("WARNING - disconnected cluster", clidx)
                success = False
                break
            for v in subg[k]:
                new_subg[rekey_dict[k]][rekey_dict[v]] = 1
                new_subg[rekey_dict[v]][rekey_dict[k]] = 1
        if not success:
            continue
        assert(len(new_subg) == n)
        assert(len(new_subg[0]) == n)
        steps = test_convergence(new_subg, 0.01)
        print("Cluster", clidx, "of size: ", n, "took n steps to converge:", steps)
        import math
        print("steps/ log2 nodes", steps/math.log2(n))
        """
        n = len(graph)
        trivial_subg = [[0 for _ in range(n)] for _ in range(n)]
        for i, k in enumerate(graph):
            for j, _ in enumerate(graph[k]):
                trivial_subg[i][j] = 1
        steps = test_convergence(trivial_subg, 0.01)
        print("Whole graph took n steps to converge:", steps)
        import math
        print("steps/ log2 nodes", steps/math.log2(n))
        """

test()


n: 54
e 288
start iterate
eigenvecs sample [1.        +0.j 0.94362863+0.j]
first two eigvenvals [1.        +0.j 0.94362863+0.j]
dist 0.013326939693194761
Cluster 0 of size:  54 took n steps to converge: 54
steps/ log2 nodes 9.38332851505776
n: 54
e 284
start iterate
eigenvecs sample [1.        +0.j 0.94135545+0.j]
first two eigvenvals [1.        +0.j 0.94135545+0.j]
dist 0.012667771267272388
Cluster 1 of size:  54 took n steps to converge: 54
steps/ log2 nodes 9.38332851505776
n: 54
e 285
start iterate
eigenvecs sample [1.        +0.j 0.94646405+0.j]
first two eigvenvals [1.        +0.j 0.94646405+0.j]
dist 0.012840286646083528
Cluster 2 of size:  54 took n steps to converge: 54
steps/ log2 nodes 9.38332851505776
n: 70
e 442
start iterate
eigenvecs sample [1.        +0.j 0.91475455+0.j]
first two eigvenvals [1.        +0.j 0.91475455+0.j]
dist 0.009758065391866529
Cluster 3 of size:  70 took n steps to converge: 38
steps/ log2 nodes 6.19974634797341
n: 54
e 288
start iterate
eigenvecs 

In [28]:
run_decomp(["synthetic/cluster_graph_50_15.graph"])

synthetic/cluster_graph_50_15.graph
real 0.63
user 1.32
sys 0.00
n clusters found 5


FileNotFoundError: [Errno 2] No such file or directory: 'synthetic/cluster_graph_50_15.graph.part.5'